In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/credit-record/credit_record.csv


# 1. Data manipulation and flagging

In [4]:
# Importing the credit record for analyzing the last 12 months default
credit_df = pd.read_csv('/kaggle/input/credit-record/credit_record.csv',delimiter=';')

# Sorting decrescenting as 0 is the most recent date and -11 is the past 12 month
credit_df.sort_values(['ID','MONTHS_BALANCE'],
                      ascending=[True, False])
# Getting the last 12 months to create the 
last_12 = credit_df[credit_df['MONTHS_BALANCE'].between(-11,0)]

# Considering that the default is accumulative, so if it has default90, it also have default 60 and 30.
# Status description: 
# C = Paid default
# X = No default at the moment
# 0 = 0 to 29 days of default
# 1 = 30 to 59 days of default
# 2 = 60 to 89 days of default
# 3 = 90 to 119 days of default
# 4 = 120 to 149 days of default
# 5 = bad debt or more than 150 days of default
flag_df = (last_12.assign(default30=last_12['STATUS'].isin(['1','2','3','4','5']))
           .assign(default60=last_12['STATUS'].isin(['2','3','4','5']))
           .assign(default90=last_12['STATUS'].isin(['3','4','5']))
           .assign(default120=last_12['STATUS'].isin(['4','5'])))

# Testing the result
flag_df[flag_df['default90']==True]


,ID,MONTHS_BALANCE,STATUS,default30,default60,default90,default120
70719,5004891,0,5,True,True,True,True
77253,5005205,0,5,True,True,True,True
77254,5005205,-1,5,True,True,True,True
77255,5005205,-2,5,True,True,True,True
104474,5009524,0,3,True,True,True,False
...,...,...,...,...,...,...,...
1040950,5149834,-7,5,True,True,True,True
1040951,5149834,-8,5,True,True,True,True
1040952,5149834,-9,5,True,True,True,True
1040953,5149834,-10,5,True,True,True,True


In [9]:
# Exporting for analyzing
flag_df.to_csv('credit_record_flagged.csv')
